# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
import numpy as np
import pandas as pd

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [2]:
# Your code here:
employee = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
employee.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
employee.isna().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
employee.groupby('Salary or Hourly').agg({'Name':'count'})

,Name
Salary or Hourly,
Hourly,8022
Salary,25161


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
employee.groupby('Department').agg({'Name':'count'})

,Name
Department,
ADMIN HEARNG,39
ANIMAL CONTRL,81
AVIATION,1629
BOARD OF ELECTION,107
BOARD OF ETHICS,8
BUDGET & MGMT,46
BUILDINGS,269
BUSINESS AFFAIRS,171
CITY CLERK,84


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

* $H_0$: the hourly workes earn 30 dollars an hour.
* $H_1$: the hourly workers doesn't earn 30 dollars an hour.

In [16]:
import scipy.stats as st
data_sample = employee[employee['Hourly Rate'].isnull() == False]
sample = data_sample['Hourly Rate']
tStat, pValue = st.ttest_1samp(sample,30)
print("P-Value:{0} T-Statistic:{1}".format(pValue,tStat))

P-Value:4.3230240486229894e-92 T-Statistic:20.6198057854942


Conclusion:

--> For a 95% of two sided confidence interval, we have 0.05 of significance. So p-value is less than our significance and because of that we reject the null hypothesis ($H_0$).

p-value < $\alpha$

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

* $H_0$: the police department annual salary is equal to $86000.
* $H_1$: the police department annual salary is greater than to $86000. (one-sided test)

In [39]:
data_department = employee[(employee['Department']=='POLICE')]
data_department.groupby(['Department','Salary or Hourly']).agg({'Name':'count'}).reset_index()

,Department,Salary or Hourly,Name
0,POLICE,Hourly,10
1,POLICE,Salary,13404


* Here we have 2 types of employees in police department. I'm going to considering only the employees who have Salary. Because the hourly employees can be appear as an outlier in the data.

In [52]:
department = data_department[data_department['Salary or Hourly']=='Salary']
data_department.shape[0] - department.shape[0] # checking if the 10 rows for hourly employees were excluded

10

In [53]:
sample_police = department['Annual Salary']
tStat, pValue = st.ttest_1samp(sample_police,86000)
print("P-Value:{0} T-Statistic:{1}".format(pValue/2,tStat))

P-Value:0.0010301701775482569 T-Statistic:3.081997005712994


Conclusion:

--> For a 95% of one sided confidence interval, we have 0.05 of significance.

p-value < $\alpha$

So, we reject the null hypothesis ($H_0$).

Using the `crosstab` function, find the department that has the most hourly workers. 

In [101]:
department_hourly = pd.crosstab(employee['Department'], employee['Salary or Hourly'])#.reset_index()
department_hourly.loc[department_hourly['Hourly']==department_hourly['Hourly'].max()]

Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

* $H_0$: the STREETS & SAN department hourly wage is equal to 35.
* $H_1$: the STREETS & SAN department hourly wage is less than 35.. (one-sided test)

In [104]:
hourly_department = employee[(employee['Department']=='STREETS & SAN')]
hourly_department.groupby(['Department','Salary or Hourly']).agg({'Name':'count'}).reset_index()

,Department,Salary or Hourly,Name
0,STREETS & SAN,Hourly,1862
1,STREETS & SAN,Salary,336


In [107]:
hourly_streets = hourly_department[hourly_department['Salary or Hourly']=='Hourly']
hourly_department.shape[0] - hourly_streets.shape[0] # checking if the 336 rows for salary employees were excluded

336

In [108]:
sample_hourly = hourly_streets['Hourly Rate']
tStat, pValue = st.ttest_1samp(sample_hourly,35)
print("P-Value:{0} T-Statistic:{1}".format(pValue/2,tStat))

P-Value:1.6689265282353859e-21 T-Statistic:-9.567447887848152


Conclusion:

--> For a 95% of one sided confidence interval and 0.05 of significance:

p-value < $\alpha$

So, we reject the null hypothesis ($H_0$).

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [ ]:
# Your code here:



Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [ ]:
# Your code here:



# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

